<a href="https://colab.research.google.com/github/kuroneko913/lab/blob/master/RSA%E6%9A%97%E5%8F%B7%E3%82%92%E3%82%84%E3%81%A3%E3%81%A6%E3%81%BF%E3%81%9F(%E5%8B%95%E4%BD%9C%E7%A2%BA%E8%AA%8D%E6%B8%88%E3%81%BF).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

RSA暗号 を実装する

In [0]:
from numpy import random
import math
import base64

In [0]:
# ユークリッドの互除法により、最大公約数(GCD)を求める
def euclid_r_gcd(a,b):
  if (a>b):
    n = a
    m = b
  else:
    n = b
    m = a
  if (m==0): return n
  else:
    print(f'{n}%{m}:{n%m},{n}-{int(n/m)}*{m}={n%m}')
    return euclid_r_gcd(m,n%m)

In [0]:
# ax+by=gcd(a,b)となるx,y,gcd(a,b)を求める(拡張ユークリッドの互除法)
def ext_euclid_gcd(z1,z2,x1=1,y1=0,x2=0,y2=1):
  if z2==0: return x1,y1,z1
  q = z1//z2
  x = x1 - q * x2
  y = y1 - q * y2
  z = z1 - q * z2
  return ext_euclid_gcd(z2,z,x2,y2,x,y)

In [0]:
# a,bの最小公倍数はa*b/gcd(a,b)
def lcm(a,b):
  return a*b/ext_euclid_gcd(a,b)[-1]

In [0]:
# 素数判定法のアルゴリズム
def is_prime(n:int)->bool:
  limit = int(math.sqrt(n))+1
  if n == 2: return True
  if n==0 or n == 1 or n%2==0 : return False
  for i in range(3,limit,2):
    if n%i==0: return False
  return True

In [6]:
euclid_r_gcd(7,5)

7%5:2,7-1*5=2
5%2:1,5-2*2=1
2%1:0,2-2*1=0


1

In [9]:
ext_euclid_gcd(7,5)

(-2, 3, 1)

In [10]:
ext_euclid_gcd(84,11)

(-3, 23, 1)

In [11]:
print(lcm(5,32))
print(ext_euclid_gcd(5,32))

160.0
(13, -2, 1)


公開鍵と秘密鍵を生成する

In [12]:
# 巨大な素数の組の生成
gcd = 10
num_length = 14 # 計算精度の問題か、20以上だとうまく復号化できないことがある
prime_p = False
prime_q = False
count = 0
while (gcd !=1 or prime_p is False or prime_q is False):
  count += 1
  print(f'try count:{count}')
  if (prime_p is False ): 
    p = random.randint(2**num_length/2)
    prime_p = is_prime(p)
  if (prime_q is False): 
    q = random.randint(2**num_length/2)
    prime_q = is_prime(q)
  gcd = ext_euclid_gcd(p,q)[-1]
  

try count:1
try count:2
try count:3
try count:4
try count:5
try count:6
try count:7
try count:8
try count:9
try count:10
try count:11
try count:12
try count:13
try count:14
try count:15
try count:16
try count:17
try count:18


In [13]:
# 公開鍵
e = 65537
n = p*q
print(e,n)

65537 22965119


In [14]:
print(p,q)
is_prime(q)

5669 4051


True

In [15]:
# 秘密鍵を生成する
phi_m = lcm(p-1,q-1)
ext_euclid_gcd_result = ext_euclid_gcd(e,phi_m)
d = abs(int(ext_euclid_gcd_result[0]))
print(abs(d))

2595473


In [0]:
# 暗号化 & 復号化
plain_text = 'Hello,RSA!'
ord_plain_text = list(map(ord,plain_text))
encode = lambda x,e,n: list(map(lambda value: pow(value,e,n),x))

In [17]:
# 暗号化
print(ord_plain_text)
codes = encode(ord_plain_text,e,n)
print(codes)

[72, 101, 108, 108, 111, 44, 82, 83, 65, 33]
[9821214, 21072138, 6971869, 6971869, 6572834, 4465620, 11496652, 6076682, 19298445, 13912503]


In [18]:
# 復号化
decodes = encode(codes, d, n)
print(decodes)
print(''.join(list(map(chr,decodes))))

[72, 101, 108, 108, 111, 44, 82, 83, 65, 33]
Hello,RSA!


In [19]:
# デジタル署名もやってみる
m = "This messsage is written by myblackcat7112 2019.08.29 08:50 "
ord_m = list(map(ord,m))
print(ord_m)

[84, 104, 105, 115, 32, 109, 101, 115, 115, 115, 97, 103, 101, 32, 105, 115, 32, 119, 114, 105, 116, 116, 101, 110, 32, 98, 121, 32, 109, 121, 98, 108, 97, 99, 107, 99, 97, 116, 55, 49, 49, 50, 32, 50, 48, 49, 57, 46, 48, 56, 46, 50, 57, 32, 48, 56, 58, 53, 48, 32]


In [20]:
# 自分の持っている秘密鍵で暗号化
encoded_message = encode(ord_m,d,n)
print(' '.join(map(str,encoded_message)))

922598 10179898 17318198 990985 12809622 6260581 1120419 990985 990985 990985 392818 14416348 1120419 12809622 17318198 990985 12809622 10418436 3964669 17318198 15090455 15090455 1120419 4436864 12809622 6518167 3143990 12809622 6260581 3143990 6518167 16188935 392818 538330 1070615 538330 392818 15090455 1579865 631524 631524 8725313 12809622 8725313 2674557 631524 9245371 19241437 2674557 10494576 19241437 8725313 9245371 12809622 2674557 10494576 12394140 21780922 2674557 12809622


In [21]:
# 公開鍵で復号するとメッセージが読める
decoded_message = encode(encoded_message,e,n)
print(''.join(list(map(chr,decoded_message))))

This messsage is written by myblackcat7112 2019.08.29 08:50 


In [22]:
e,n

(65537, 22965119)

In [23]:
# せっかくなので、base64に変換して、それっぽくする
str_message = ' '.join(map(str,encoded_message)).encode('utf-8')
base64_coded = base64.b64encode(str_message)
v_ = ''
v__ = []
for v in base64_coded:
  if (len(v_)==76):
    v__.append(v_)
    v_ =''
  v_ += chr(v)
v__.append(v_)

# base64で表示
for v in v__:
  print(v)

OTIyNTk4IDEwMTc5ODk4IDE3MzE4MTk4IDk5MDk4NSAxMjgwOTYyMiA2MjYwNTgxIDExMjA0MTkg
OTkwOTg1IDk5MDk4NSA5OTA5ODUgMzkyODE4IDE0NDE2MzQ4IDExMjA0MTkgMTI4MDk2MjIgMTcz
MTgxOTggOTkwOTg1IDEyODA5NjIyIDEwNDE4NDM2IDM5NjQ2NjkgMTczMTgxOTggMTUwOTA0NTUg
MTUwOTA0NTUgMTEyMDQxOSA0NDM2ODY0IDEyODA5NjIyIDY1MTgxNjcgMzE0Mzk5MCAxMjgwOTYy
MiA2MjYwNTgxIDMxNDM5OTAgNjUxODE2NyAxNjE4ODkzNSAzOTI4MTggNTM4MzMwIDEwNzA2MTUg
NTM4MzMwIDM5MjgxOCAxNTA5MDQ1NSAxNTc5ODY1IDYzMTUyNCA2MzE1MjQgODcyNTMxMyAxMjgw
OTYyMiA4NzI1MzEzIDI2NzQ1NTcgNjMxNTI0IDkyNDUzNzEgMTkyNDE0MzcgMjY3NDU1NyAxMDQ5
NDU3NiAxOTI0MTQzNyA4NzI1MzEzIDkyNDUzNzEgMTI4MDk2MjIgMjY3NDU1NyAxMDQ5NDU3NiAx
MjM5NDE0MCAyMTc4MDkyMiAyNjc0NTU3IDEyODA5NjIy


In [24]:
# base64をdecodeしてから、デジタル署名メッセージが復元されることを確認
sign_message=''.join(v__)
base64_decoded_codes = map(int,base64.b64decode(sign_message).decode('utf-8').split(' '))
# メッセージと公開鍵さえあれば、復元できる！
print(''.join(map(chr,encode(base64_decoded_codes,e,n))))

This messsage is written by myblackcat7112 2019.08.29 08:50 
